# Lecture-1: Overview and Tokenization

In the first lecture of CS-336 we mainly studied about **tokenization** and with the focus of mainly at **BPE (Byte-Pair Encoding) tokenizer**. 

## 1. Intro To Tokenization
